In [1]:
#imports
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns

from PathFinder import find_path

In [2]:
#read data
PATH = find_path()

#select files
#files = ['application_train.csv','application_test.csv',]
files = ['preprocessed_train.csv', 'preprocessed_test.csv']
"""         
          'bureau.csv', 
         'bureau_balance.csv', 'credit_card_balance.csv',
         'installments_payments.csv', 'POS_CASH_balance.csv',
         'previous_application.csv']
"""
data_files=[]
data=[]
print("files used: {}".format(files))

#actually read files
for file in files:
    stuff = pd.read_csv(PATH+file,header=0,engine='python')
    #print(stuff)
    data_files.append(file)
    data.append(stuff)

path: data/
all files in directory: ['application_test.csv', 'application_train.csv', 'bureau.csv', 'bureau_balance.csv', 'credit_card_balance.csv', 'HomeCredit_columns_description.csv', 'installments_payments.csv', 'POS_CASH_balance.csv', 'preprocessed_test.csv', 'preprocessed_train.csv', 'previous_application.csv', 'sample_submission.csv']
files used: ['preprocessed_train.csv', 'preprocessed_test.csv']


In [3]:
#select and convert columns
original_train = data[0] #save the original because we'll need some stuff later
original_test = data[1]
app_train = data[0].copy()
app_test = data[1].copy()

#delete ID cols
del app_train['SK_ID_CURR']
del app_test['SK_ID_CURR']

app_train_y = app_train['TARGET']
new_cols_x = app_train.columns.values.tolist()
new_cols_x.remove('TARGET')
app_train_x = app_train[new_cols_x]

print('Training Features shape: ', app_train_x.shape) #241 cols
print('Testing Features shape: ', app_test.shape) #241 cols

Training Features shape:  (307511, 235)
Testing Features shape:  (48744, 235)


In [5]:
#neural net train
from sklearn.neural_network import MLPRegressor
#mlp = MLPRegressor(hidden_layer_sizes=(300,250,200,150,100,50,40,30,20,10), #possibly too deep lol
mlp = MLPRegressor(hidden_layer_sizes=(300,200,100,50,10), #possibly too deep lol
#                   max_iter=1000)
                   max_iter=100, random_state=20)
mlp.fit(app_train_x, app_train_y)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(300, 200, 100, 50, 10),
       learning_rate='constant', learning_rate_init=0.001, max_iter=100,
       momentum=0.9, nesterovs_momentum=True, power_t=0.5, random_state=20,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [6]:
# Make predictions
mlp_pred = mlp.predict(app_test)
mlp_pred = mlp_pred.clip(0, 1)

In [7]:
# Submission dataframe
submit = original_test[['SK_ID_CURR']]
submit['TARGET'] = mlp_pred

submit.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,SK_ID_CURR,TARGET
0,100001,0.030802
1,100005,0.152246
2,100013,0.055726
3,100028,0.030592
4,100038,0.184220


In [8]:
# Save the submission to a csv file
overwrite = True
if overwrite:
    submit.to_csv('results/mlp_baseline_2.csv', index = False)

Submitted to kaggle  
  
mlp_baseline.csv was created with MLPRegressor(hidden_layer_sizes=(300,250,200,150,100,50,40,30,20,10),max_iter=1000)  
score: 0.73529  
  
mlp_baseline.csv was created with MLPRegressor(hidden_layer_sizes=(300,200,100,50,10),max_iter=100,random_state=20)  
score: 0.70327